In [35]:
import pandas as pd
import seaborn as sns
sns.set_theme(style="white")


import pprint

import os
import sys


sys.path.append(os.path.join(os.path.abspath(''), '..'))
import utils.file_utils as ut
from gdc.gdc_api import GDCSchema

pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)


In [36]:
df_target = pd.read_csv('../data/target.csv')
df_target.drop('study', axis=1, inplace=True)
print(df_target.shape)
df_target.head(5)

(1068, 16)


,case_submitter_id,age_at_diagnosis,race,ethnicity,gender,vital_status,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_stage,tumor_grade,tumor_focality,tumor_largest_dimension_diameter,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_code
0,01BR001,20089.0,black or african american,not hispanic or latino,female,Alive,T2,N1c,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA
1,01BR008,17532.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
2,01BR009,23376.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
3,01BR010,23741.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
4,01BR015,12784.0,white,not hispanic or latino,female,Alive,T2,N1,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA


### GEt the categories and variables in GDC for the target data

In [41]:
interested_subscheams = ['clinical', 'demographic', 'diagnosis',  'pathology_detail', 'sample']  
interested_cols = set(df_target.columns) 
interested_cols.remove('case_submitter_id')

gdc_interested_column =  {}

covered_cols = set()
for subschema in interested_subscheams:
    print('Checking', subschema)
    for col in interested_cols:
        # print(col, len(df_target[col].unique()))
        sc = GDCSchema(col, subschema)
        candidate = subschema + '::' + col
        properties = sc.get_properties_by_gdc_candidate(candidate)
        if isinstance(properties, dict) and 'enum' in properties:
            #print(col, properties['enum'])
            print(col)
            covered_cols.add(col)
        elif isinstance(properties, dict) and 'type' in properties:
            print(col, properties)
            covered_cols.add(col)    
    print('\n')
print('Covered', covered_cols)
print('non-covered', interested_cols - covered_cols)

Checking clinical
vital_status ['alive', 'dead', 'lost to follow-up']
ethnicity ['hispanic or latino', 'not hispanic or latino']
age_at_diagnosis {'description': 'Age in days at which a condition or disease was first diagnosed. CDE: 3225640. TCGA has the age in years with CDE 2006657, this is converted by ceil(age*365.25) to get days.', 'type': 'number'}
gender ['female', 'male', 'unknown', 'unspecified']
race ['not reported', 'white', 'american indian or alaska native', 'black or african american', 'asian', 'native hawaiian or other pacific islander', 'other']


Checking demographic
vital_status ['Alive', 'Dead', 'Unknown', 'Not Reported']
ethnicity ['hispanic or latino', 'not hispanic or latino', 'Unknown', 'unknown', 'not reported', 'not allowed to collect']
gender ['female', 'male', 'unspecified', 'unknown', 'not reported']
race ['american indian or alaska native', 'asian', 'black or african american', 'native hawaiian or other pacific islander', 'white', 'other', 'Unknown', 'unkno

### Sample category is not included in the description, but it appears in target in tumor code

In [38]:
sc = GDCSchema('tumor_code')
print(sc.get_gdc_candidates())
sc = GDCSchema('tumor_largest_dimension_diameter')
print(sc.get_gdc_candidates())

{'sample::tumor_code': {'score': 1, 'description': 'Diagnostic tumor code of the tissue sample source.'}, 'sample::tumor_code_id': {'score': 0.923076923076923, 'description': 'BCR-defined id code for the tumor sample.'}, 'diagnosis::tumor_depth': {'score': 0.8424242424242424, 'description': 'Numeric value that represents the depth of tumor invasion, measured in millimeters (mm).'}, 'diagnosis::tumor_grade': {'score': 0.8424242424242424, 'description': 'Numeric value to express the degree of abnormality of cancer cells, a measure of differentiation and aggressiveness.'}, 'aligned_reads::tumor_ploidy': {'score': 0.8222222222222223, 'description': 'Numeric value used to describe the number of sets of chromosomes in a cell or an organism. For example, haploid means one set and diploid means two sets.'}, 'diagnosis::tumor_burden': {'score': 0.8222222222222223, 'description': 'Refers to the number of cancer cells, the size of a tumor, or the amount of cancer in the body.'}, 'sample::tumor_de

In [39]:
ground_truth_path = os.path.join( '..', 'data','table-matching-ground-truth', 'ground-truth')

groundtruth_target_cols = set()

for file in os.listdir(ground_truth_path):

    study = file.split('.csv')[0]
    
    groundtruth_path = os.path.join(ground_truth_path, file)
    groundtruth_col_pairs = ut.load_table_matching_groundtruth(groundtruth_path)

    target_cols = [entry[1] for entry in groundtruth_col_pairs]
    groundtruth_target_cols.update(target_cols)
    print(study, target_cols)



Krug ['ajcc_pathologic_stage', 'age_at_diagnosis', 'gender', 'ethnicity']
Clark ['case_submitter_id', 'tissue_type', 'gender', 'age_at_diagnosis', 'bmi', 'race', 'ethnicity', 'site_of_resection_or_biopsy;tissue_or_organ_of_origin', 'tumor_largest_dimension_diameter', 'tumor_focality', 'primary_diagnosis', 'tumor_grade', 'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n']
Vasaikar ['Case_ID', 'age_at_diagnosis', 'gender', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_stage', 'vascular_invasion_present', 'lymphatic_invasion_present', 'perineural_invasion_present', 'synchronous_malignancy', 'vital_status', 'last_known_disease_status']
Gillette ['case_submitter_id', 'tissue_type', 'can be inferred from tobacco_smoking_status', 'age_at_diagnosis', 'gender', 'ethnicity', 'height', 'weight', 'bmi', 'cigarettes_per_day', 'pack_years_smoked', 'tobacco_smoking_status', 'secondhand_smoke_as_child', 'primary_diagnosis']
Wang ['case_submitter_id', 'age_at_diagnosis', 'ge

In [40]:
additional_cols = groundtruth_target_cols - interested_cols
additional_cols.remove('case_submitter_id')
additional_cols.remove('Case_ID')
print('Number of Columns in the Groundtruth (as target) that are not in the Target (GDC prepared data): ', len(additional_cols))

for col in additional_cols:
    if ';' in col:
        cols = col.split(';')
        for c in cols:
            c = c.strip()
            sc = GDCSchema(c)
            candidates = sc.get_gdc_candidates()
            filtered_candidates = [entry for entry in candidates if candidates[entry]['score'] == 1]
            print(c,' ', filtered_candidates)
    else:
        sc = GDCSchema(col)
        candidates = sc.get_gdc_candidates()
        filtered_candidates = [entry for entry in candidates if candidates[entry]['score'] == 1]
        print(col,' ', filtered_candidates)
        
        #print(col, sc.get_gdc_candidates())
        #print('\n')
    

#Columns in the Groundtruth (as target) that are not in the Target (GDC prepared data) 32
tobacco_smoking_quit_year   ['exposure::tobacco_smoking_quit_year']
last_known_disease_status   ['diagnosis::last_known_disease_status']
cigarettes_per_day   ['exposure::cigarettes_per_day']
weight   ['other_clinical_attribute::weight', 'portion::weight', 'follow_up::weight']
perineural_invasion_present   ['pathology_detail::perineural_invasion_present']
lymph_nodes_positive   ['pathology_detail::lymph_nodes_positive']
ajcc_clinical_m   ['diagnosis::ajcc_clinical_m']
vascular_invasion_present   ['pathology_detail::vascular_invasion_present']
lymphatic_invasion_present   ['pathology_detail::lymphatic_invasion_present']
tobacco_smoking_status   ['exposure::tobacco_smoking_status']
tobacco_smoking_onset_year   ['exposure::tobacco_smoking_onset_year']
tissue_type   ['sample::tissue_type']
ajcc_pathologic_m   ['diagnosis::ajcc_pathologic_m']
progression_or_recurrence   ['follow_up::progression_or_recur